# Causal Injection into a Neural Network

In [18]:
save_figs = True
if save_figs:
    import kaleido

## Reconstruction experiment

### Recostruction Experiment 20% known

In [19]:
rerun_experiment_20=False
if rerun_experiment_20:
    ## Run main script
    %run -i main_biggerDAG_extended.py --version="r_ex3"  --branchf=1 --known_p=0.2 --noise_p=0.0
    %run -i main_biggerDAG_extended.py --version="r_ex_2b"  --branchf=2 --known_p=0.2 --noise_p=0.0
    %run -i main_biggerDAG_extended.py --version="r_ex_5b"  --branchf=5 --known_p=0.2 --noise_p=0.0

In [20]:
import re
import os
import numpy as np
import pandas as pd
from utils import load_pickle, plot_ly_by

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b

version = '(r_ex3|r_ex_2b|r_ex_5b)'

known_p = 0.2

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )
# display(df)

df = df[df['alpha']!=1000]

    
plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", save=save_figs, name='NV', version='1l_32h_20k_0n', y2_vis=False, showleg=False)

# plot_ly_by(df, 'N_nodes', x_desc="|V| = Number of Nodes in G")
# plot_ly_by(df[df['alpha'] == 100], 'N_nodes', x_desc="|V| = Number of Nodes in G")
plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", save=save_figs, name='N100', version='1l_32h_20k_0n', y1_vis=False)

plot_ly_by(df, 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$",  y2_ticks=[0.1,0.3,0.5,0.7],
save=save_figs, name='N', version='1l_32h_20k_0n', y2_vis=False, showleg=False, y2_range= [0, 1.8])    

# plot_ly_by(df, 'alpha3', x_desc="Dataset Size / b (b=N Edges/N Nodes)")
plot_ly_by(df, 'alpha5', x_desc=r"$N/|E| \text{ (Dataset Size / N Edges)}$", save=save_figs, name='NE', version='1l_32h_20k_0n',y2_vis=False, showleg=False
# y1_vis=False, legend_cord = [0.1,1]
, y1_range=[-0.8, 1.1], y2_ticks=[0,0.2,0.4,0.6]
)

# plot_ly_by(df, 'branch', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
plot_ly_by(df, 'branch_bin', x_desc=r"$b = |E|/|V| \text{ (Number of Edges / Number of Nodes)}$", save=save_figs, name='b', version='1l_32h_20k_0n', y1_vis=False
# y2_vis=False, showleg=False
, y1_range=[-0.8, 1.1], y2_ticks=[0,0.2,0.4,0.6]
)



# plot_ly_by('Size', df[df['N_nodes'] == 10], x_desc="Dataset Size")

### Check How many runs you have of a given version
# v="r_ex_5b"
# print((df[df['V']==v].groupby(by=['seed']).size()))
# print(df[df['V']==v].groupby(by=['N_nodes']).size())
# print(df[df['V']==v].groupby(by=['alpha']).size())

Count= 48


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,30.12,16.02,90,51.03,51.69,90,0.17,0.22,0.62 (0.43)
1,-1.00,100,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,60.61,31.93,90,51.03,51.69,90,0.34,0.23,0.53 (0.36)
2,-1.00,200,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,121.60,63.66,90,51.03,51.69,90,0.54,0.22,0.5 (0.33)
3,-1.00,300,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,182.43,95.41,90,51.03,51.69,90,0.54,0.23,0.5 (0.33)
4,-1.00,500,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,304.42,158.88,90,51.03,51.69,90,0.55,0.23,0.49 (0.33)
5,0.05,50,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,30.12,16.02,90,51.03,51.69,90,0.27,0.26,0.6 (0.42)
6,0.05,100,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,60.61,31.93,90,51.03,51.69,90,0.51,0.23,0.51 (0.33)
7,0.05,200,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,121.60,63.66,90,51.03,51.69,90,0.57,0.23,0.5 (0.33)
8,0.05,300,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,182.43,95.41,90,51.03,51.69,90,0.56,0.24,0.5 (0.33)
9,0.05,500,90,26.67,17.09,90,63.28,64.64,90,2811,...,90,304.42,158.88,90,51.03,51.69,90,0.58,0.22,0.5 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,60,19.47,9.06,60,2811,2490.52,60,937.5,...,60,47.85,28.78,60,16.0,7.36,60,0.27,0.26,0.5 (0.39)
1,-1.00,20,60,45.60,25.74,60,2811,2490.52,60,1875.0,...,60,44.88,29.75,60,36.9,20.68,60,0.10,0.12,0.62 (0.43)
2,-1.00,50,60,124.77,75.80,60,2811,2490.52,60,4687.5,...,60,43.37,30.17,60,100.2,60.57,60,0.39,0.23,0.6 (0.37)
3,0.05,10,60,19.47,9.06,60,2811,2490.52,60,937.5,...,60,47.85,28.78,60,16.0,7.36,60,0.43,0.29,0.49 (0.39)
4,0.05,20,60,45.60,25.74,60,2811,2490.52,60,1875.0,...,60,44.88,29.75,60,36.9,20.68,60,0.27,0.26,0.59 (0.41)
5,0.05,50,60,124.77,75.80,60,2811,2490.52,60,4687.5,...,60,43.37,30.17,60,100.2,60.57,60,0.47,0.22,0.57 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,150,19.47,9.01,150,2811,2477.95,150,2875,...,150,147.47,132.79,150,16.0,7.33,150,0.47,0.30,0.46 (0.36)
1,-1.00,20,150,45.60,25.61,150,2811,2477.95,150,5750,...,150,138.45,132.01,150,36.9,20.57,150,0.35,0.28,0.55 (0.37)
2,-1.00,50,150,124.77,75.42,150,2811,2477.95,150,14375,...,150,133.59,131.38,150,100.2,60.27,150,0.46,0.22,0.57 (0.34)
3,0.05,10,150,19.47,9.01,150,2811,2477.95,150,2875,...,150,147.47,132.79,150,16.0,7.33,150,0.56,0.28,0.46 (0.36)
4,0.05,20,150,45.60,25.61,150,2811,2477.95,150,5750,...,150,138.45,132.01,150,36.9,20.57,150,0.43,0.27,0.54 (0.36)
5,0.05,50,150,124.77,75.42,150,2811,2477.95,150,14375,...,150,133.59,131.38,150,100.2,60.27,150,0.50,0.21,0.56 (0.32)


,Type,alpha5,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0.0, 50.0]",128,28.36,17.29,128,92.81,79.38,128,2839.06,...,128,29.55,13.52,128,74.64,63.47,128,0.23,0.23,0.54 (0.39)
1,-1.00,"(50.0, 100.0]",97,24.54,16.46,97,60.31,62.45,97,2753.71,...,97,69.41,18.43,97,48.65,49.93,97,0.34,0.21,0.52 (0.37)
2,-1.00,"(100.0, 200.0]",123,26.26,17.05,123,59.38,58.20,123,2831.87,...,123,140.06,34.96,123,47.94,46.54,123,0.52,0.23,0.5 (0.33)
3,-1.00,"(200.0, inf]",102,27.06,17.15,102,33.74,25.93,102,2805.10,...,102,344.95,112.40,102,27.40,20.80,102,0.65,0.22,0.55 (0.34)
4,0.05,"(0.0, 50.0]",128,28.36,17.29,128,92.81,79.38,128,2839.06,...,128,29.55,13.52,128,74.64,63.47,128,0.31,0.23,0.52 (0.38)
5,0.05,"(50.0, 100.0]",97,24.54,16.46,97,60.31,62.45,97,2753.71,...,97,69.41,18.43,97,48.65,49.93,97,0.47,0.23,0.51 (0.36)
6,0.05,"(100.0, 200.0]",123,26.26,17.05,123,59.38,58.20,123,2831.87,...,123,140.06,34.96,123,47.94,46.54,123,0.57,0.21,0.5 (0.33)
7,0.05,"(200.0, inf]",102,27.06,17.15,102,33.74,25.93,102,2805.10,...,102,344.95,112.40,102,27.40,20.80,102,0.67,0.22,0.55 (0.34)


,Type,branch_bin,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0, 1]",150,26.67,17.05,150,26.20,16.89,150,2811,...,150,235.14,164.48,150,21.27,13.55,150,0.52,0.31,0.6 (0.39)
1,-1.00,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811,...,150,124.13,87.39,150,41.10,27.45,150,0.42,0.25,0.56 (0.33)
2,-1.00,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811,...,150,60.24,44.25,150,90.73,66.82,150,0.34,0.22,0.43 (0.33)
3,0.05,"(0, 1]",150,26.67,17.05,150,26.20,16.89,150,2811,...,150,235.14,164.48,150,21.27,13.55,150,0.61,0.28,0.59 (0.38)
4,0.05,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811,...,150,124.13,87.39,150,41.10,27.45,150,0.51,0.23,0.55 (0.33)
5,0.05,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811,...,150,60.24,44.25,150,90.73,66.82,150,0.37,0.21,0.42 (0.32)


In [21]:
df["perfect"] = (df['right']==1).astype(int)
df[df['alpha']==500][["V","Type","perfect"]].groupby(["V","Type"]).agg([ 'count','mean','std'])

perfect                    
                count      mean       std
V       Type                             
r_ex3   -1.00      30  0.033333  0.182574
         0.05      30  0.033333  0.182574
r_ex_2b -1.00      30  0.000000  0.000000
         0.05      30  0.000000  0.000000
r_ex_5b -1.00      30  0.000000  0.000000
         0.05      30  0.000000  0.000000

### Recostruction Experiment 50% known

In [22]:
rerun_experiment_50=False
if rerun_experiment_50:
    ## Run main script
    %run -i main_biggerDAG_extended.py --version="r_ex_1b_50"  --branchf=2 --known_p=0.5 --noise_p=0.0
    %run -i main_biggerDAG_extended.py --version="r_ex_5b_50"  --branchf=5 --known_p=0.5 --noise_p=0.0

In [23]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b 20%known
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b 20%known
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b 20%known
# # 'r_ex_1b_50' = reconstruction experiment, 10n 20n 50n 2b 50%known
version = '(r_ex_1b_50|r_ex_5b_50)'

known_p = 0.5

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 150), (150, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
# display(df)


plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", save=save_figs, name='NV', version='1l_32h_5k_0n', y2_vis=False, showleg=False)

# plot_ly_by(df, 'N_nodes', x_desc="|V| = Number of Nodes in G")
# plot_ly_by(df[df['alpha'] == 100], 'N_nodes', x_desc="|V| = Number of Nodes in G")
plot_ly_by(df[df['alpha'] <= 50], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", save=save_figs, name='N100', version='1l_32h_50k_0n', y1_vis=False)


## Following, not new info
# plot_ly_by(df, 'branch', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
plot_ly_by(df, 'branch_bin', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")

# plot_ly_by(df, 'alpha3', x_desc="Dataset Size / b (b=N Edges/N Nodes)")
plot_ly_by(df, 'alpha5', x_desc="N/|E| (Dataset Size / N Edges)")

# plot_ly_by('Size', df[df['N_nodes'] == 10], x_desc="Dataset Size")

Count= 30


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,19.68,7.32,60,41.17,35.73,60,0.19,0.27,0.57 (0.41)
1,-1.00,100,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,39.80,14.52,60,41.17,35.73,60,0.46,0.33,0.49 (0.33)
2,-1.00,200,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,80.17,29.18,60,41.17,35.73,60,0.72,0.23,0.47 (0.31)
3,-1.00,300,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,120.30,43.58,60,41.17,35.73,60,0.73,0.24,0.47 (0.31)
4,-1.00,500,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,200.98,72.67,60,41.17,35.73,60,0.72,0.24,0.46 (0.32)
5,0.05,50,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,19.68,7.32,60,41.17,35.73,60,0.44,0.37,0.55 (0.39)
6,0.05,100,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,39.80,14.52,60,41.17,35.73,60,0.75,0.23,0.48 (0.32)
7,0.05,200,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,80.17,29.18,60,41.17,35.73,60,0.79,0.24,0.47 (0.32)
8,0.05,300,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,120.30,43.58,60,41.17,35.73,60,0.77,0.23,0.47 (0.32)
9,0.05,500,60,26.67,17.14,60,81.82,71.48,60,2811,...,60,200.98,72.67,60,41.17,35.73,60,0.81,0.22,0.47 (0.32)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,20,24.35,7.21,20,2811,2533.83,20,625,...,20,21.95,6.55,20,12.5,3.59,20,0.25,0.35,0.5 (0.4)
1,-1.00,20,20,58.60,22.25,20,2811,2533.83,20,1250,...,20,19.25,7.57,20,29.5,11.16,20,0.01,0.02,0.61 (0.43)
2,-1.00,50,20,162.50,66.52,20,2811,2533.83,20,3125,...,20,17.85,7.56,20,81.5,33.26,20,0.31,0.23,0.62 (0.39)
3,0.05,10,20,24.35,7.21,20,2811,2533.83,20,625,...,20,21.95,6.55,20,12.5,3.59,20,0.67,0.35,0.5 (0.41)
4,0.05,20,20,58.60,22.25,20,2811,2533.83,20,1250,...,20,19.25,7.57,20,29.5,11.16,20,0.08,0.08,0.58 (0.42)
5,0.05,50,20,162.50,66.52,20,2811,2533.83,20,3125,...,20,17.85,7.56,20,81.5,33.26,20,0.57,0.28,0.56 (0.33)


,Type,branch_bin,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0, 1]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)
1,-1.00,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811.0,...,150,124.13,87.39,150,25.63,17.12,150,0.62,0.36,0.56 (0.34)
2,-1.00,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811.0,...,150,60.24,44.25,150,56.70,41.82,150,0.51,0.31,0.43 (0.33)
3,0.05,"(0, 1]",0,NaN,NaN,0,NaN,NaN,0,NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,nan (nan)
4,0.05,"(1, 2]",150,26.67,17.05,150,50.77,34.27,150,2811.0,...,150,124.13,87.39,150,25.63,17.12,150,0.78,0.30,0.56 (0.34)
5,0.05,"(2, 5]",150,26.67,17.05,150,112.87,83.68,150,2811.0,...,150,60.24,44.25,150,56.70,41.82,150,0.64,0.28,0.42 (0.32)


,Type,alpha5,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,"(0.0, 50.0]",110,29.00,17.34,110,104.00,80.00,110,2838.18,...,110,26.20,11.51,110,52.26,39.99,110,0.34,0.32,0.51 (0.37)
1,-1.00,"(50.0, 100.0]",67,23.58,16.12,67,75.58,69.12,67,2728.06,...,67,64.22,12.63,67,38.04,34.54,67,0.54,0.29,0.46 (0.33)
2,-1.00,"(100.0, 150.0]",54,28.52,17.09,54,87.35,73.32,54,2808.33,...,54,111.94,9.81,54,43.93,36.66,54,0.72,0.24,0.49 (0.32)
3,-1.00,"(150.0, 200.0]",39,22.82,16.69,39,46.18,31.52,39,2880.51,...,39,161.92,9.74,39,23.36,15.74,39,0.83,0.20,0.45 (0.31)
4,-1.00,"(200.0, inf]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,270.37,16.18,30,25.63,17.35,30,0.82,0.22,0.53 (0.32)
5,0.05,"(0.0, 50.0]",110,29.00,17.34,110,104.00,80.00,110,2838.18,...,110,26.20,11.51,110,52.26,39.99,110,0.54,0.32,0.5 (0.36)
6,0.05,"(50.0, 100.0]",67,23.58,16.12,67,75.58,69.12,67,2728.06,...,67,64.22,12.63,67,38.04,34.54,67,0.74,0.24,0.46 (0.32)
7,0.05,"(100.0, 150.0]",54,28.52,17.09,54,87.35,73.32,54,2808.33,...,54,111.94,9.81,54,43.93,36.66,54,0.78,0.23,0.5 (0.33)
8,0.05,"(150.0, 200.0]",39,22.82,16.69,39,46.18,31.52,39,2880.51,...,39,161.92,9.74,39,23.36,15.74,39,0.89,0.19,0.46 (0.32)
9,0.05,"(200.0, inf]",30,26.67,17.29,30,50.77,34.74,30,2811.00,...,30,270.37,16.18,30,25.63,17.35,30,0.93,0.14,0.54 (0.33)


## Noise Experiment

Input random noise variables that are not part of the causal graph into the dataset

In [48]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability, plot_ly_by

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b

version = '(r_ex_1b_k20_l1_h3_n20|r_ex_2b_k20_l1_h3_n20|r_ex_5b_k20_l1_h3_n20)'

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching','fooled','fooled_e']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching','fooled','fooled_e']
df
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)
df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
df

# v="r_ex_2b_k20_l1_h3_n20"

# print((df[df['V']==v].groupby(by=['seed']).size()))
# print(df[df['V']==v].groupby(by=['N_nodes']).size())
# print(df[df['V']==v].groupby(by=['alpha']).size())

v="r_ex_1b_k20_l1_h3_n20"

print(v)
print((df[df['V']==v].groupby(by=['seed']).size()))
print(df[df['V']==v].groupby(by=['N_nodes']).size())
print(df[df['V']==v].groupby(by=['alpha']).size())

v="r_ex_5b_k20_l1_h3_n20"

print(v)
print((df[df['V']==v].groupby(by=['seed']).size()))
print(df[df['V']==v].groupby(by=['N_nodes']).size())
print(df[df['V']==v].groupby(by=['alpha']).size())

Count= 44
r_ex_1b_k20_l1_h3_n20
seed
0       30
10      30
100     30
1000    24
dtype: int64
N_nodes
10    40
20    40
50    34
dtype: int64
alpha
50     24
100    24
200    22
300    22
500    22
dtype: int64
r_ex_5b_k20_l1_h3_n20
seed
0    28
dtype: int64
N_nodes
10    10
20    10
50     8
dtype: int64
alpha
50     6
100    6
200    6
300    6
500    4
dtype: int64


In [25]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability, plot_ly_by

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b

version = '(r_ex_2b_k20_l1_h3_n20)'

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching','fooled','fooled_e']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching','fooled','fooled_e']
df
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)
df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*0.2).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
df


plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", save=save_figs, name='NV', version='1l_32h_20k_2n', y2_vis=False, showleg=False)

plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", save=save_figs, name='N100', version='1l_32h_20k_2n', y1_vis=False)


# plot_ly_by(df, x='alpha', x_desc="s = N/|V| (Dataset Size / Number of Nodes)", 
#             y1='fooled', y1_desc='% of edges from/to noise variables', y1_range=[-0.1,0.15], y1_ticks=[0,0.1])
# plot_ly_by(df, x='N_nodes', x_desc="|V| = Number of Nodes in G", 
#             y1='fooled', y1_desc='% of edges from/to noise variables', y1_range=[-0.1,0.15], y1_ticks=[0,0.1])

plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", 
            y1='fooled', y1_desc='% of edges from/to noise variables', y1_range=[-0.1,0.12], y1_ticks=[0,0.1], y2_ticks=[0.1,0.3,0.5,0.7],
            save=save_figs, name='NVnoise', version='1l_32h_20k_2n', y2_vis=False, showleg=False)

# plot_ly_by(df, 'N_nodes', x_desc="|V| = Number of Nodes in G")
# plot_ly_by(df[df['alpha'] == 100], 'N_nodes', x_desc="|V| = Number of Nodes in G")
plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$" ,
            y1='fooled', y1_desc='% of edges from/to noise variables', y1_range=[-0.1,0.12], y1_ticks=[0,0.1], y2_ticks=[0.1,0.3,0.5,0.7],
            save=save_figs, name='N100noise', version='1l_32h_20k_2n', y1_vis=False)

# plot_ly_by(df, 'N_nodes', x_desc="|V| = Number of Nodes in G")
# plot_ly_by(df[df['alpha'] == 100], 'N_nodes', x_desc="|V| = Number of Nodes in G")

# plot_ly_by(df, 'branch', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
# plot_ly_by(df, 'branch_bin', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")

# plot_ly_by(df, 'alpha3', x_desc="Dataset Size / b (b=N Edges/N Nodes)")
# plot_ly_by(df, 'alpha5', x_desc="N/|E| (Dataset Size / N Edges)")
# v="r_ex_2b_k20_l1_h3_n20"

# print((df[df['V']==v].groupby(by=['seed']).size()))
# print(df[df['V']==v].groupby(by=['N_nodes']).size())
# print(df[df['V']==v].groupby(by=['alpha']).size())

Count= 15


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,666.67,432.18,30,41.1,27.83,30,0.11,0.15,0.67 (0.41)
1,-1.00,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,1333.33,864.36,30,41.1,27.83,30,0.22,0.24,0.57 (0.35)
2,-1.00,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,2666.67,1728.73,30,41.1,27.83,30,0.55,0.19,0.53 (0.31)
3,-1.00,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,4000.00,2593.09,30,41.1,27.83,30,0.53,0.19,0.53 (0.31)
4,-1.00,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,6666.67,4321.82,30,41.1,27.83,30,0.57,0.19,0.53 (0.32)
5,0.05,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,666.67,432.18,30,41.1,27.83,30,0.19,0.21,0.66 (0.41)
6,0.05,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,1333.33,864.36,30,41.1,27.83,30,0.33,0.21,0.55 (0.33)
7,0.05,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,2666.67,1728.73,30,41.1,27.83,30,0.58,0.20,0.54 (0.32)
8,0.05,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,4000.00,2593.09,30,41.1,27.83,30,0.56,0.16,0.53 (0.32)
9,0.05,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,6666.67,4321.82,30,41.1,27.83,30,0.58,0.19,0.54 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,375,128.25,20,14.5,0.95,20,0.07,0.09,0.67 (0.42)
1,-1.00,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,750,256.49,20,30.1,1.07,20,0.02,0.03,0.55 (0.39)
2,-1.00,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,1875,641.24,20,78.7,1.13,20,0.41,0.17,0.65 (0.32)
3,0.05,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,375,128.25,20,14.5,0.95,20,0.18,0.17,0.66 (0.43)
4,0.05,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,750,256.49,20,30.1,1.07,20,0.11,0.13,0.52 (0.38)
5,0.05,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,1875,641.24,20,78.7,1.13,20,0.48,0.16,0.63 (0.3)


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,666.67,432.18,30,41.1,27.83,30,0.11,0.15,0.67 (0.41)
1,-1.00,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,1333.33,864.36,30,41.1,27.83,30,0.22,0.24,0.57 (0.35)
2,-1.00,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,2666.67,1728.73,30,41.1,27.83,30,0.55,0.19,0.53 (0.31)
3,-1.00,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,4000.00,2593.09,30,41.1,27.83,30,0.53,0.19,0.53 (0.31)
4,-1.00,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,6666.67,4321.82,30,41.1,27.83,30,0.57,0.19,0.53 (0.32)
5,0.05,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,666.67,432.18,30,41.1,27.83,30,0.19,0.21,0.66 (0.41)
6,0.05,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,1333.33,864.36,30,41.1,27.83,30,0.33,0.21,0.55 (0.33)
7,0.05,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,2666.67,1728.73,30,41.1,27.83,30,0.58,0.20,0.54 (0.32)
8,0.05,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,4000.00,2593.09,30,41.1,27.83,30,0.56,0.16,0.53 (0.32)
9,0.05,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,6666.67,4321.82,30,41.1,27.83,30,0.58,0.19,0.54 (0.33)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha2count,alpha2mean,alpha2std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,375,128.25,20,14.5,0.95,20,0.07,0.09,0.67 (0.42)
1,-1.00,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,750,256.49,20,30.1,1.07,20,0.02,0.03,0.55 (0.39)
2,-1.00,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,1875,641.24,20,78.7,1.13,20,0.41,0.17,0.65 (0.32)
3,0.05,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,375,128.25,20,14.5,0.95,20,0.18,0.17,0.66 (0.43)
4,0.05,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,750,256.49,20,30.1,1.07,20,0.11,0.13,0.52 (0.38)
5,0.05,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,1875,641.24,20,78.7,1.13,20,0.48,0.16,0.63 (0.3)


## Multi-Layer Experiment

### 3 hidden layers: 2 x n_input - n_input x 2/3 - 2 x n_input

In [26]:
rerun_multi_2=False
if rerun_multi_2:
    ## Run main script
    %run -i main_biggerDAG_extended.py --version="r_ex_2b_k20_l3_h2_n0" --branchf=2 --known_p=0.2 --noise_p=0.0 --hidden_l=3 --hidden_n_p=2

In [27]:
import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b 20%known
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b 20%known
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b 20%known
# # 'r_ex_1b_50' = reconstruction experiment, 10n 20n 50n 2b 50%known
version = '(r_ex_2b_k20_l3_h2_n0)'

known_p = 0.2

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 150), (150, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
# display(df)

plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", 
                save=save_figs, name='NV', version='3l_2h_20k_0n', y2_vis=False, showleg=False, y2_range= [0, 2])

plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", 
                save=save_figs, name='N100', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 2])    

plot_ly_by(df, 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$",  y2_ticks=[0.1,0.3,0.5,0.7],
            save=save_figs, name='N', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 1.8] )    


### Not useful to repeat for this experiment
# plot_ly_by(df, 'branch', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")
# plot_ly_by(df, 'branch_bin', x_desc="b = |E|/|V| (Number of Edges / Number of Nodes)")

# plot_ly_by(df, 'alpha3', x_desc="Dataset Size / b (b=N Edges/N Nodes)")
# plot_ly_by(df, 'alpha5', x_desc="N/|E| (Dataset Size / N Edges)")


# plot_ly_by('Size', df[df['N_nodes'] == 10], x_desc="Dataset Size")

Count= 15


,Type,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,26.67,1.60,30,41.1,27.83,30,0.11,0.16,0.81 (0.48)
1,-1.00,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,53.67,3.12,30,41.1,27.83,30,0.28,0.22,0.57 (0.34)
2,-1.00,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,108.07,6.34,30,41.1,27.83,30,0.50,0.19,0.54 (0.31)
3,-1.00,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,161.90,9.70,30,41.1,27.83,30,0.50,0.20,0.53 (0.31)
4,-1.00,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,270.37,16.18,30,41.1,27.83,30,0.58,0.16,0.53 (0.32)
5,0.05,50,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,26.67,1.60,30,41.1,27.83,30,0.24,0.21,0.68 (0.43)
6,0.05,100,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,53.67,3.12,30,41.1,27.83,30,0.48,0.18,0.55 (0.33)
7,0.05,200,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,108.07,6.34,30,41.1,27.83,30,0.57,0.20,0.53 (0.31)
8,0.05,300,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,161.90,9.70,30,41.1,27.83,30,0.55,0.17,0.53 (0.31)
9,0.05,500,30,26.67,17.29,30,50.77,34.74,30,2811,...,30,270.37,16.18,30,41.1,27.83,30,0.59,0.16,0.53 (0.32)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,42.40,14.77,20,14.5,0.95,20,0.27,0.21,0.71 (0.39)
1,-1.00,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,40.05,13.86,20,30.1,1.07,20,0.14,0.16,0.59 (0.49)
2,-1.00,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,38.05,13.20,20,78.7,1.13,20,0.18,0.24,0.76 (0.41)
3,0.05,10,20,17.5,0.95,20,2811,2533.83,20,937.5,...,20,42.40,14.77,20,14.5,0.95,20,0.36,0.25,0.69 (0.44)
4,0.05,20,20,37.1,1.07,20,2811,2533.83,20,1875.0,...,20,40.05,13.86,20,30.1,1.07,20,0.24,0.22,0.53 (0.41)
5,0.05,50,20,97.7,1.13,20,2811,2533.83,20,4687.5,...,20,38.05,13.20,20,78.7,1.13,20,0.48,0.14,0.62 (0.3)


,Type,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,Sizemean,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,10,50,17.5,0.93,50,2811,2494.75,50,2875,...,50,131.30,93.07,50,14.5,0.93,50,0.48,0.26,0.63 (0.37)
1,-1.00,20,50,37.1,1.05,50,2811,2494.75,50,5750,...,50,123.68,87.29,50,30.1,1.05,50,0.34,0.24,0.49 (0.38)
2,-1.00,50,50,97.7,1.11,50,2811,2494.75,50,14375,...,50,117.42,82.71,50,78.7,1.11,50,0.36,0.24,0.66 (0.34)
3,0.05,10,50,17.5,0.93,50,2811,2494.75,50,2875,...,50,131.30,93.07,50,14.5,0.93,50,0.52,0.27,0.62 (0.39)
4,0.05,20,50,37.1,1.05,50,2811,2494.75,50,5750,...,50,123.68,87.29,50,30.1,1.05,50,0.42,0.24,0.47 (0.34)
5,0.05,50,50,97.7,1.11,50,2811,2494.75,50,14375,...,50,117.42,82.71,50,78.7,1.11,50,0.51,0.13,0.61 (0.28)


## Compare to 1Layer

In [28]:


import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b 20%known
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b 20%known
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b 20%known
# # 'r_ex_1b_50' = reconstruction experiment, 10n 20n 50n 2b 50%known
version = '(r_ex_2b|r_ex_2b_k20_l3_h2_n0)'

known_p = 0.2

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 150), (150, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
display(df)

# plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", 
#                 save=save_figs, name='NV', version='3l_2h_20k_0n', y2_vis=False, showleg=False, y2_range= [0, 2])

# plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", 
#                 save=save_figs, name='N100', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 2])    

# plot_ly_by(df, 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$",  y2_ticks=[0.1,0.3,0.5,0.7],
#             save=save_figs, name='N', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 1.8] )    



Count= 30


,V,Type,N_nodes,N_edges,seed,Size,MSE,MAE,right,matching,alpha,branch,branch_bin,alpha2,alpha3,alpha4,alpha5,rebase_den,rebased
0,r_ex_2b,-1.00,10,16,0,2500,0.930440,0.769476,0.875000,14,200,2,"(1, 2]",1000,"(500.0, 1000.0]",125,"(100.0, 150.0]",13,1.000000
1,r_ex_2b,0.05,10,16,0,2500,0.924337,0.766918,0.687500,11,200,2,"(1, 2]",1000,"(500.0, 1000.0]",125,"(100.0, 150.0]",13,0.846154
2,r_ex_2b,-1.00,10,17,10,2500,0.667189,0.652212,0.176471,3,200,2,"(1, 2]",1000,"(500.0, 1000.0]",117,"(100.0, 150.0]",14,0.214286
3,r_ex_2b,0.05,10,17,10,2500,0.654310,0.645737,0.235294,4,200,2,"(1, 2]",1000,"(500.0, 1000.0]",117,"(100.0, 150.0]",14,0.285714
4,r_ex_2b,-1.00,10,19,100,2500,0.767214,0.696095,0.736842,14,200,2,"(1, 2]",1000,"(500.0, 1000.0]",105,"(100.0, 150.0]",16,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,r_ex_2b,0.05,10,17,5000,6250,0.104036,0.256399,0.411765,7,500,2,"(1, 2]",2500,"(2000.0, 5000.0]",294,"(200.0, inf]",14,0.500000
596,r_ex_2b,-1.00,10,17,6000,6250,0.173949,0.330382,0.176471,3,500,2,"(1, 2]",2500,"(2000.0, 5000.0]",294,"(200.0, inf]",14,0.214286
597,r_ex_2b,0.05,10,17,6000,6250,0.173000,0.329343,0.588235,10,500,2,"(1, 2]",2500,"(2000.0, 5000.0]",294,"(200.0, inf]",14,0.714286
598,r_ex_2b,-1.00,10,18,7000,6250,0.206830,0.360777,0.777778,14,500,2,"(1, 2]",2500,"(2000.0, 5000.0]",277,"(200.0, inf]",15,0.933333


In [29]:
main_gray = '#262626'
sec_gray = '#595959'
main_blue = '#005383'
sec_blue = '#0085CA'


# df=df[df['alpha'] <= 100]
x='alpha'
x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$"
y1='right'
y1_desc="% of right edges"
y1_range=[-0.8, 1]
y1_ticks=[0,0.2,0.4,0.6,0.8,1]
y1_vis=True
y2='MSEmean'
y2_desc="Mean Squared Error"
y2_range= [0, 1.8]
y2_ticks=[0,0.2,0.5,0.8]
y2_vis=True
showleg = True
save=True
name='compalpha'
version='1lvs3l'
xwidth = 1100
names_list = ['CASTLE+ 3L','CASTLE+ 1L','Injected 1L','Injected 3L']
# names_list = ['CASTLE+ w/noise','CASTLE+','Injected','Injected w/noise']
colors_list = [sec_gray,main_gray,main_blue,sec_blue]

df = df.sort_values(by=[x],axis=0)
# df=df[['Type', 'N_nodes','alpha', 'MSE', 'MAE','right']]
mses = df.groupby(['Type','V',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
mses.columns = list(map(''.join, mses.columns.values))
mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)
mses[x] = mses[x].astype(str)

print(np.unique(df['V']))
display(mses)

fig = make_subplots(specs=[[{"secondary_y": True}]])

########## BarCharts
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[0],#'#FF4136', 
    marker_line_color=colors_list[0],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[1],#'#FF4136', 
    marker_line_color=colors_list[1],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)


msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[2],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[2],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)
msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[3],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[3],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)


########## Boxplots
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[0],
    marker_color= colors_list[0]
),
secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[1],
    marker_color=colors_list[1]#'#FF4136'
),
    secondary_y=False,)


if y1=='right':
    d = msesC['rebasedmean']#[i*1.2 for i in msesC['rightmean']]

    fig.add_trace(
        go.Scatter(x=msesC[x], y=d, name="CASTLE+20%" , mode='markers', marker_symbol='line-ew',
                marker=dict(
                color='Black',
                size=50,
                line=dict(
                    color='Black',
                    width=2)
                    ),
                showlegend=False),
        secondary_y=False, 
    )


fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[2],
    marker_color=colors_list[2]#'#0085CA'#'#3D9970'
),
    secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[3],
    marker_color= colors_list[3]
),
    secondary_y=False,)


fig.update_layout(
    # yaxis_title='normalized moisture',
    boxmode='group' # group together boxes of the different traces for each value of x
    ,bargap=0.2
    ,bargroupgap=0.1
)

if not y1_vis:
    y1_desc = ''
if not y2_vis:
    y2_desc = ''
# Set x-axis title
fig.update_xaxes(showgrid=True,
title={'text':x_desc#,'font':{'size':18}
})
# Set y-axes titles
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y1_desc#,'font':{'size':18}
}, 
range=y1_range,
tickvals=y1_ticks,
tickformat=".0%",
secondary_y=False,
showticklabels=y1_vis
)
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y2_desc#,'font':{'size':18}
},
range=y2_range,
tickvals=y2_ticks,
secondary_y=True,
showticklabels=y2_vis)

if save:
    # Add figure title
    fig.update_layout(
        showlegend=showleg,
        title='',
        legend={
            'y':1.1,
            'x':0.5,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'top'},
        template='plotly_white',
        autosize=True,
        width=xwidth, height=350, 
        margin=dict(
            l=10,
            r=10,
            b=0,
            t=10,
            pad=0
        ),
        font=dict(
            family='Serif',#"Courier New, monospace",
            size=18,
            color="Black"
        )    
    )

    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    out_path = os.path.join(output_folder,f"plot_{name}_{version}.png")

    import kaleido
    fig.write_image(out_path)

else:
    # Add figure title
    fig.update_layout(
        margin=dict(l=50, r=0, t=20, b=20),
        title={
            'text': "",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend={
            # 'y':-0.17,
            # 'x':0.85,
            # 'orientation':"h",
            # 'xanchor': 'center',
            # 'yanchor': 'bottom'},
            'y':1.05,
            'x':0.8,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'bottom'},
        template='plotly_white',
            font=dict(
                family='Serif',#"Courier New, monospace",
                size=18,
                color="Black"
            )
    )

fig.show()

['r_ex_2b' 'r_ex_2b_k20_l3_h2_n0']


,Type,V,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,r_ex_2b,50,30,26.67,17.29,30,50.77,34.74,30,...,30,26.67,1.60,30,41.1,27.83,30,0.13,0.15,0.64 (0.39)
1,-1.00,r_ex_2b,100,30,26.67,17.29,30,50.77,34.74,30,...,30,53.67,3.12,30,41.1,27.83,30,0.33,0.21,0.55 (0.33)
2,-1.00,r_ex_2b,200,30,26.67,17.29,30,50.77,34.74,30,...,30,108.07,6.34,30,41.1,27.83,30,0.57,0.19,0.53 (0.31)
3,-1.00,r_ex_2b,300,30,26.67,17.29,30,50.77,34.74,30,...,30,161.90,9.70,30,41.1,27.83,30,0.53,0.20,0.54 (0.32)
4,-1.00,r_ex_2b,500,30,26.67,17.29,30,50.77,34.74,30,...,30,270.37,16.18,30,41.1,27.83,30,0.56,0.17,0.53 (0.32)
5,-1.00,r_ex_2b_k20_l3_h2_n0,50,30,26.67,17.29,30,50.77,34.74,30,...,30,26.67,1.60,30,41.1,27.83,30,0.11,0.16,0.81 (0.48)
6,-1.00,r_ex_2b_k20_l3_h2_n0,100,30,26.67,17.29,30,50.77,34.74,30,...,30,53.67,3.12,30,41.1,27.83,30,0.28,0.22,0.57 (0.34)
7,-1.00,r_ex_2b_k20_l3_h2_n0,200,30,26.67,17.29,30,50.77,34.74,30,...,30,108.07,6.34,30,41.1,27.83,30,0.50,0.19,0.54 (0.31)
8,-1.00,r_ex_2b_k20_l3_h2_n0,300,30,26.67,17.29,30,50.77,34.74,30,...,30,161.90,9.70,30,41.1,27.83,30,0.50,0.20,0.53 (0.31)
9,-1.00,r_ex_2b_k20_l3_h2_n0,500,30,26.67,17.29,30,50.77,34.74,30,...,30,270.37,16.18,30,41.1,27.83,30,0.58,0.16,0.53 (0.32)


In [30]:
main_gray = '#262626'
sec_gray = '#595959'
main_blue = '#005383'
sec_blue = '#0085CA'


# df=df[df['alpha'] <= 100]
x='N_nodes'
x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$"
y1='right'
y1_desc="% of right edges"
y1_range=[-0.8, 1]
y1_ticks=[0,0.2,0.4,0.6,0.8,1]
y1_vis=True
y2='MSEmean'
y2_desc="Mean Squared Error"
y2_range= [0, 1.8]
y2_ticks=[0,0.2,0.5,0.8]
y2_vis=True
showleg = True
save=True
name='compnodes'
version='1lvs3l'
xwidth = 1100
names_list = ['CASTLE+ 3L','CASTLE+ 1L','Injected 1L','Injected 3L']
# names_list = ['CASTLE+ w/noise','CASTLE+','Injected','Injected w/noise']
colors_list = [sec_gray,main_gray,main_blue,sec_blue]

df = df.sort_values(by=[x],axis=0)
# df=df[['Type', 'N_nodes','alpha', 'MSE', 'MAE','right']]
mses = df.groupby(['Type','V',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
mses.columns = list(map(''.join, mses.columns.values))
mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)
mses[x] = mses[x].astype(str)

print(np.unique(df['V']))
display(mses)

fig = make_subplots(specs=[[{"secondary_y": True}]])

########## BarCharts
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[0],#'#FF4136', 
    marker_line_color=colors_list[0],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[1],#'#FF4136', 
    marker_line_color=colors_list[1],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)


msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[2],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[2],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)
msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[3],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[3],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)


########## Boxplots
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[0],
    marker_color= colors_list[0]
),
secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[1],
    marker_color=colors_list[1]#'#FF4136'
),
    secondary_y=False,)


if y1=='right':
    d = msesC['rebasedmean']#[i*1.2 for i in msesC['rightmean']]

    fig.add_trace(
        go.Scatter(x=msesC[x], y=d, name="CASTLE+20%" , mode='markers', marker_symbol='line-ew',
                marker=dict(
                color='Black',
                size=50,
                line=dict(
                    color='Black',
                    width=2)
                    ),
                showlegend=False),
        secondary_y=False, 
    )


fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[2],
    marker_color=colors_list[2]#'#0085CA'#'#3D9970'
),
    secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[3],
    marker_color= colors_list[3]
),
    secondary_y=False,)


fig.update_layout(
    # yaxis_title='normalized moisture',
    boxmode='group' # group together boxes of the different traces for each value of x
    ,bargap=0.2
    ,bargroupgap=0.1
)

if not y1_vis:
    y1_desc = ''
if not y2_vis:
    y2_desc = ''
# Set x-axis title
fig.update_xaxes(showgrid=True,
title={'text':x_desc#,'font':{'size':18}
})
# Set y-axes titles
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y1_desc#,'font':{'size':18}
}, 
range=y1_range,
tickvals=y1_ticks,
tickformat=".0%",
secondary_y=False,
showticklabels=y1_vis
)
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y2_desc#,'font':{'size':18}
},
range=y2_range,
tickvals=y2_ticks,
secondary_y=True,
showticklabels=y2_vis)

if save:
    # Add figure title
    fig.update_layout(
        showlegend=showleg,
        title='',
        legend={
            'y':1.1,
            'x':0.5,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'top'},
        template='plotly_white',
        autosize=True,
        width=xwidth, height=350, 
        margin=dict(
            l=10,
            r=10,
            b=0,
            t=10,
            pad=0
        ),
        font=dict(
            family='Serif',#"Courier New, monospace",
            size=18,
            color="Black"
        )    
    )

    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    out_path = os.path.join(output_folder,f"plot_{name}_{version}.png")

    import kaleido
    fig.write_image(out_path)

else:
    # Add figure title
    fig.update_layout(
        margin=dict(l=50, r=0, t=20, b=20),
        title={
            'text': "",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend={
            # 'y':-0.17,
            # 'x':0.85,
            # 'orientation':"h",
            # 'xanchor': 'center',
            # 'yanchor': 'bottom'},
            'y':1.05,
            'x':0.8,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'bottom'},
        template='plotly_white',
            font=dict(
                family='Serif',#"Courier New, monospace",
                size=18,
                color="Black"
            )
    )

fig.show()

['r_ex_2b' 'r_ex_2b_k20_l3_h2_n0']


,Type,V,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,r_ex_2b,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.43,0.31,0.59 (0.37)
1,-1.00,r_ex_2b,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.36,0.25,0.46 (0.31)
2,-1.00,r_ex_2b,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.48,0.16,0.62 (0.3)
3,-1.00,r_ex_2b_k20_l3_h2_n0,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.48,0.26,0.63 (0.37)
4,-1.00,r_ex_2b_k20_l3_h2_n0,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.34,0.24,0.49 (0.38)
5,-1.00,r_ex_2b_k20_l3_h2_n0,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.36,0.24,0.66 (0.34)
6,0.05,r_ex_2b,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.57,0.29,0.59 (0.37)
7,0.05,r_ex_2b,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.43,0.24,0.46 (0.32)
8,0.05,r_ex_2b,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.53,0.12,0.6 (0.28)
9,0.05,r_ex_2b_k20_l3_h2_n0,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.52,0.27,0.62 (0.39)


## Compare to Noise

In [31]:


import re
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import load_pickle, random_stability

verbose = False
verbose2 = False
nodes = '([\w]+)'
seeds = '([\w]+)'
dsets = '([\w.]+)'
outfs = str(1)
infs = str(1)
thetas = '([-+]?([0-9]+)(\.[0-9]+)?)'

# ## Versions
# # 'v3' reconstrunction toy experiment - 84%
# # 'p1' = partial toy experiment - 96%
# # 'r2' = reconstruction toy experiment, 20n 4b - 0%
# # 'r3' = reconstruction toy experiment, 10n 4b - 0%
# # 'r4' = reconstruction toy experiment, 10n 1b - 0%
# # 'r5' = reconstruction toy experiment, 15n 2b - ?
# # 'r6' = reconstruction toy experiment, 10n 1b - 32hidden
# # 'r7' = reconstruction toy experiment, 20n 4b - 0%
# # 'r_ex3' = reconstruction experiment, 10n 20n 50n 1b 20%known
# # 'r_ex_2b' = reconstruction experiment, 10n 20n 50n 2b 20%known
# # 'r_ex_5b' = reconstruction experiment, 10n 20n 50n 5b 20%known
# # 'r_ex_1b_50' = reconstruction experiment, 10n 20n 50n 2b 50%known
version = '(r_ex_2b|r_ex_2b_k20_l1_h3_n20)'

known_p = 0.2

folder = './results/'
# CASTLE/results/Nested1FoldCASTLE.Reg.Synth.10.625.r_ex2.pkl

count = 0
res = []

for filename in os.listdir(folder):
    match = re.search(f'Nested1FoldCASTLE.Reg.Synth.{nodes}.{dsets}.{version}.pkl$', filename)
    if match != None:
        count += 1
        
        report = load_pickle(os.path.join(folder, filename), verbose=False)
        for key, value in report.items():
            # print(key, value)
            t = [[match.group(3)],[value[i] for i in ['theta', 'n_nodes', 'N_edges', 'seed', 'data_size', 'MSE', 'MAE','right','matching']]]
            res.append([i for sublist in t for i in sublist])
            # print(res)
print("Count=",count)

df = pd.DataFrame(res)
df.columns = ['V','Type', 'N_nodes', 'N_edges', 'seed', 'Size', 'MSE', 'MAE','right','matching']
df['alpha'] = ((df['Size'].astype(int)*0.8)/df['N_nodes'].astype(int)).astype(int)
df['branch'] = np.round(((df['N_edges'].astype(int))/df['N_nodes'].astype(int)).astype(float),0).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 1), (1, 2), (2, 5)])
df['branch_bin'] = pd.cut(df['branch'], bins)

df['alpha2'] = ((df['Size'].astype(int)*0.8)/df['branch'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 500), (500, 1000), (1000, 2000), (2000, 5000), (5000,np.Inf)])
df['alpha3'] = pd.cut(df['alpha2'], bins)

df['alpha4'] = ((df['Size'].astype(int)*0.8)/df['N_edges'].astype(int)).astype(int)
bins = pd.IntervalIndex.from_tuples([(0, 50), (50, 100), (100, 150), (150, 200), (200,np.Inf)])
df['alpha5'] = pd.cut(df['alpha4'], bins)

df['rebase_den'] = (df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))
df['rebased'] = (df['matching'].astype(int)/(df['N_edges'].astype(int)-(df['N_edges']*known_p).astype(int))).clip(upper= 1 )

df = df[df['alpha'] != 1000]
display(df)

# plot_ly_by(df, 'alpha', x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$", 
#                 save=save_figs, name='NV', version='3l_2h_20k_0n', y2_vis=False, showleg=False, y2_range= [0, 2])

# plot_ly_by(df[df['alpha'] <= 100], 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$", 
#                 save=save_figs, name='N100', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 2])    

# plot_ly_by(df, 'N_nodes', x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$",  y2_ticks=[0.1,0.3,0.5,0.7],
#             save=save_figs, name='N', version='3l_2h_20k_0n', y1_vis=False, y2_range= [0, 1.8] )    



Count= 30


,V,Type,N_nodes,N_edges,seed,Size,MSE,MAE,right,matching,alpha,branch,branch_bin,alpha2,alpha3,alpha4,alpha5,rebase_den,rebased
0,r_ex_2b_k20_l1_h3_n20,-1.00,20,37,0,1250,1.163986,0.863243,0.000000,0,50,2,"(1, 2]",500,"(0.0, 500.0]",27,"(0.0, 50.0]",30,0.000000
1,r_ex_2b_k20_l1_h3_n20,0.05,20,37,0,1250,1.155092,0.852532,0.027027,1,50,2,"(1, 2]",500,"(0.0, 500.0]",27,"(0.0, 50.0]",30,0.033333
2,r_ex_2b_k20_l1_h3_n20,-1.00,20,36,10,1250,0.293958,0.425193,0.000000,0,50,2,"(1, 2]",500,"(0.0, 500.0]",27,"(0.0, 50.0]",29,0.000000
3,r_ex_2b_k20_l1_h3_n20,0.05,20,36,10,1250,0.240622,0.385128,0.027778,1,50,2,"(1, 2]",500,"(0.0, 500.0]",27,"(0.0, 50.0]",29,0.034483
4,r_ex_2b_k20_l1_h3_n20,-1.00,20,38,100,1250,1.211399,0.874832,0.000000,0,50,2,"(1, 2]",500,"(0.0, 500.0]",26,"(0.0, 50.0]",31,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,r_ex_2b_k20_l1_h3_n20,0.05,20,35,5000,12500,0.216122,0.355111,0.542857,19,500,2,"(1, 2]",5000,"(2000.0, 5000.0]",285,"(200.0, inf]",28,0.678571
596,r_ex_2b_k20_l1_h3_n20,-1.00,20,38,6000,12500,0.177841,0.335577,0.368421,14,500,2,"(1, 2]",5000,"(2000.0, 5000.0]",263,"(200.0, inf]",31,0.451613
597,r_ex_2b_k20_l1_h3_n20,0.05,20,38,6000,12500,0.179548,0.337012,0.315789,12,500,2,"(1, 2]",5000,"(2000.0, 5000.0]",263,"(200.0, inf]",31,0.387097
598,r_ex_2b_k20_l1_h3_n20,-1.00,20,38,7000,12500,0.193746,0.350563,0.473684,18,500,2,"(1, 2]",5000,"(2000.0, 5000.0]",263,"(200.0, inf]",31,0.580645


In [32]:
main_gray = '#262626'
sec_gray = '#595959'
main_blue = '#005383'
sec_blue = '#0085CA'

# df=df[df['alpha'] <= 100]
x='alpha'
x_desc=r"$s = N/|V| \text{ (Dataset Size / Number of Nodes)}$"
y1='right'
y1_desc="% of right edges"
y1_range=[-0.8, 1]
y1_ticks=[0,0.2,0.4,0.6,0.8,1]
y1_vis=True
y2='MSEmean'
y2_desc="Mean Squared Error"
y2_range= [0, 1.8]
y2_ticks=[0,0.2,0.5,0.8]
y2_vis=True
showleg = True
save=True
name='compalpha'
version='1lvsNoise'
xwidth = 1100
names_list = ['CASTLE+ w/noise','CASTLE+','Injected','Injected w/noise']
colors_list = [sec_gray,main_gray,main_blue,sec_blue]

df = df.sort_values(by=[x],axis=0)
# df=df[['Type', 'N_nodes','alpha', 'MSE', 'MAE','right']]
mses = df.groupby(['Type','V',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
mses.columns = list(map(''.join, mses.columns.values))
mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)
mses[x] = mses[x].astype(str)

print(np.unique(df['V']))
display(mses)

fig = make_subplots(specs=[[{"secondary_y": True}]])

########## BarCharts
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[0],#'#FF4136', 
    marker_line_color=colors_list[0],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[1],#'#FF4136', 
    marker_line_color=colors_list[1],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)


msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[2],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[2],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)
msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[3],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[3],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)


########## Boxplots
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[0],
    marker_color= colors_list[0]
),
secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[1],
    marker_color=colors_list[1]#'#FF4136'
),
    secondary_y=False,)


if y1=='right':
    d = msesC['rebasedmean']#[i*1.2 for i in msesC['rightmean']]

    fig.add_trace(
        go.Scatter(x=msesC[x], y=d, name="CASTLE+20%" , mode='markers', marker_symbol='line-ew',
                marker=dict(
                color='Black',
                size=50,
                line=dict(
                    color='Black',
                    width=2)
                    ),
                showlegend=False),
        secondary_y=False, 
    )


fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[2],
    marker_color=colors_list[2]#'#0085CA'#'#3D9970'
),
    secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[3],
    marker_color= colors_list[3]
),
    secondary_y=False,)


fig.update_layout(
    # yaxis_title='normalized moisture',
    boxmode='group' # group together boxes of the different traces for each value of x
    ,bargap=0.2
    ,bargroupgap=0.1
)

if not y1_vis:
    y1_desc = ''
if not y2_vis:
    y2_desc = ''
# Set x-axis title
fig.update_xaxes(showgrid=True,
title={'text':x_desc#,'font':{'size':18}
})
# Set y-axes titles
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y1_desc#,'font':{'size':18}
}, 
range=y1_range,
tickvals=y1_ticks,
tickformat=".0%",
secondary_y=False,
showticklabels=y1_vis
)
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y2_desc#,'font':{'size':18}
},
range=y2_range,
tickvals=y2_ticks,
secondary_y=True,
showticklabels=y2_vis)

if save:
    # Add figure title
    fig.update_layout(
        showlegend=showleg,
        title='',
        legend={
            'y':1.1,
            'x':0.5,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'top'},
        template='plotly_white',
        autosize=True,
        width=xwidth, height=350, 
        margin=dict(
            l=10,
            r=10,
            b=0,
            t=10,
            pad=0
        ),
        font=dict(
            family='Serif',#"Courier New, monospace",
            size=18,
            color="Black"
        )    
    )

    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    out_path = os.path.join(output_folder,f"plot_{name}_{version}.png")

    import kaleido
    fig.write_image(out_path)

else:
    # Add figure title
    fig.update_layout(
        margin=dict(l=50, r=0, t=20, b=20),
        title={
            'text': "",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend={
            # 'y':-0.17,
            # 'x':0.85,
            # 'orientation':"h",
            # 'xanchor': 'center',
            # 'yanchor': 'bottom'},
            'y':1.05,
            'x':0.8,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'bottom'},
        template='plotly_white',
            font=dict(
                family='Serif',#"Courier New, monospace",
                size=18,
                color="Black"
            )
    )

fig.show()


['r_ex_2b' 'r_ex_2b_k20_l1_h3_n20']


,Type,V,alpha,N_nodescount,N_nodesmean,N_nodesstd,N_edgescount,N_edgesmean,N_edgesstd,seedcount,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,r_ex_2b,50,30,26.67,17.29,30,50.77,34.74,30,...,30,26.67,1.60,30,41.1,27.83,30,0.13,0.15,0.64 (0.39)
1,-1.00,r_ex_2b,100,30,26.67,17.29,30,50.77,34.74,30,...,30,53.67,3.12,30,41.1,27.83,30,0.33,0.21,0.55 (0.33)
2,-1.00,r_ex_2b,200,30,26.67,17.29,30,50.77,34.74,30,...,30,108.07,6.34,30,41.1,27.83,30,0.57,0.19,0.53 (0.31)
3,-1.00,r_ex_2b,300,30,26.67,17.29,30,50.77,34.74,30,...,30,161.90,9.70,30,41.1,27.83,30,0.53,0.20,0.54 (0.32)
4,-1.00,r_ex_2b,500,30,26.67,17.29,30,50.77,34.74,30,...,30,270.37,16.18,30,41.1,27.83,30,0.56,0.17,0.53 (0.32)
5,-1.00,r_ex_2b_k20_l1_h3_n20,50,30,26.67,17.29,30,50.77,34.74,30,...,30,26.67,1.60,30,41.1,27.83,30,0.11,0.15,0.67 (0.41)
6,-1.00,r_ex_2b_k20_l1_h3_n20,100,30,26.67,17.29,30,50.77,34.74,30,...,30,53.67,3.12,30,41.1,27.83,30,0.22,0.24,0.57 (0.35)
7,-1.00,r_ex_2b_k20_l1_h3_n20,200,30,26.67,17.29,30,50.77,34.74,30,...,30,108.07,6.34,30,41.1,27.83,30,0.55,0.19,0.53 (0.31)
8,-1.00,r_ex_2b_k20_l1_h3_n20,300,30,26.67,17.29,30,50.77,34.74,30,...,30,161.90,9.70,30,41.1,27.83,30,0.53,0.19,0.53 (0.31)
9,-1.00,r_ex_2b_k20_l1_h3_n20,500,30,26.67,17.29,30,50.77,34.74,30,...,30,270.37,16.18,30,41.1,27.83,30,0.57,0.19,0.53 (0.32)


In [33]:
main_gray = '#262626'
sec_gray = '#595959'
main_blue = '#005383'
sec_blue = '#0085CA'

# df=df[df['alpha'] <= 100]
x='N_nodes'
x_desc=r"$|V| = \text{ Number of Nodes in } \mathcal{G}$"
y1='right'
y1_desc="% of right edges"
y1_range=[-0.8, 1]
y1_ticks=[0,0.2,0.4,0.6,0.8,1]
y1_vis=True
y2='MSEmean'
y2_desc="Mean Squared Error"
y2_range= [0, 1.8]
y2_ticks=[0,0.2,0.5,0.8]
y2_vis=True
showleg = True
save=True
name='compnodes'
version='1lvsNoise'
xwidth = 1100
names_list = ['CASTLE+ w/noise','CASTLE+','Injected','Injected w/noise']
colors_list = [sec_gray,main_gray,main_blue,sec_blue]

df = df.sort_values(by=[x],axis=0)
# df=df[['Type', 'N_nodes','alpha', 'MSE', 'MAE','right']]
mses = df.groupby(['Type','V',x], as_index=False).agg([ 'count','mean','std']).round(2).reset_index()
mses.columns = list(map(''.join, mses.columns.values))
mses['text'] = mses[['MSEmean','MSEstd']].apply(lambda x : '{} ({})'.format(x[0],x[1]), axis=1)
mses[x] = mses[x].astype(str)

print(np.unique(df['V']))
display(mses)

fig = make_subplots(specs=[[{"secondary_y": True}]])

########## BarCharts
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[0],#'#FF4136', 
    marker_line_color=colors_list[0],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)
msesC = mses[(mses['Type']==-1) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesC[x], y=msesC[y2], name="CASTLEMSE",
    marker_color=colors_list[1],#'#FF4136', 
    marker_line_color=colors_list[1],#rgb(8,48,107)',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesC['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesC['text'],
        textposition='auto',
                ),
    secondary_y=True,
)


msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[0])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[2],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[2],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)
msesI = mses[(mses['Type']==0.05) & (mses['V']==np.unique(mses['V'])[1])]
fig.add_trace(
    go.Bar(x=msesI[x], y=msesI[y2], name="InjectedMSE",
    marker_color=colors_list[3],#'#0085CA',#'#3D9970', 
    marker_line_color=colors_list[3],#'#0085CA',#'White',
                marker_line_width=2, opacity=0.6,
    # error_y=dict(
    #         type='data', # value of error bar given in data coordinates
    #         array=msesI['MSEstd'],
    #         visible=True,
    #         color='Black',
    #         thickness=1.5,
    #         width=3),
    showlegend=False,
        text=msesI['text'],
        textposition='inside',
                ),
    secondary_y=True,
)


########## Boxplots
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[0],
    marker_color= colors_list[0]
),
secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==-1) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[1],
    marker_color=colors_list[1]#'#FF4136'
),
    secondary_y=False,)


if y1=='right':
    d = msesC['rebasedmean']#[i*1.2 for i in msesC['rightmean']]

    fig.add_trace(
        go.Scatter(x=msesC[x], y=d, name="CASTLE+20%" , mode='markers', marker_symbol='line-ew',
                marker=dict(
                color='Black',
                size=50,
                line=dict(
                    color='Black',
                    width=2)
                    ),
                showlegend=False),
        secondary_y=False, 
    )


fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[0])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[2],
    marker_color=colors_list[2]#'#0085CA'#'#3D9970'
),
    secondary_y=False,)
fig.add_trace(go.Box(
    y=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][y1],
    x=df[(df['Type']==0.05) & (df['V']==np.unique(df['V'])[1])][x].astype(str),
    boxmean='sd', # represent mean
    boxpoints=False,#'outliers',
    name=names_list[3],
    marker_color= colors_list[3]
),
    secondary_y=False,)


fig.update_layout(
    # yaxis_title='normalized moisture',
    boxmode='group' # group together boxes of the different traces for each value of x
    ,bargap=0.2
    ,bargroupgap=0.1
)

if not y1_vis:
    y1_desc = ''
if not y2_vis:
    y2_desc = ''
# Set x-axis title
fig.update_xaxes(showgrid=True,
title={'text':x_desc#,'font':{'size':18}
})
# Set y-axes titles
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y1_desc#,'font':{'size':18}
}, 
range=y1_range,
tickvals=y1_ticks,
tickformat=".0%",
secondary_y=False,
showticklabels=y1_vis
)
fig.update_yaxes(showgrid=True,nticks=10,zeroline=True, title={'text':y2_desc#,'font':{'size':18}
},
range=y2_range,
tickvals=y2_ticks,
secondary_y=True,
showticklabels=y2_vis)

if save:
    # Add figure title
    fig.update_layout(
        showlegend=showleg,
        title='',
        legend={
            'y':1.1,
            'x':0.5,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'top'},
        template='plotly_white',
        autosize=True,
        width=xwidth, height=350, 
        margin=dict(
            l=10,
            r=10,
            b=0,
            t=10,
            pad=0
        ),
        font=dict(
            family='Serif',#"Courier New, monospace",
            size=18,
            color="Black"
        )    
    )

    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    out_path = os.path.join(output_folder,f"plot_{name}_{version}.png")

    import kaleido
    fig.write_image(out_path)

else:
    # Add figure title
    fig.update_layout(
        margin=dict(l=50, r=0, t=20, b=20),
        title={
            'text': "",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend={
            # 'y':-0.17,
            # 'x':0.85,
            # 'orientation':"h",
            # 'xanchor': 'center',
            # 'yanchor': 'bottom'},
            'y':1.05,
            'x':0.8,
            # 'y':-0.08,
            # 'x':0.92,
            'orientation':"h",
            'xanchor': 'center',
            'yanchor': 'bottom'},
        template='plotly_white',
            font=dict(
                family='Serif',#"Courier New, monospace",
                size=18,
                color="Black"
            )
    )

fig.show()


['r_ex_2b' 'r_ex_2b_k20_l1_h3_n20']


,Type,V,N_nodes,N_edgescount,N_edgesmean,N_edgesstd,seedcount,seedmean,seedstd,Sizecount,...,alpha4count,alpha4mean,alpha4std,rebase_dencount,rebase_denmean,rebase_denstd,rebasedcount,rebasedmean,rebasedstd,text
0,-1.00,r_ex_2b,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.43,0.31,0.59 (0.37)
1,-1.00,r_ex_2b,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.36,0.25,0.46 (0.31)
2,-1.00,r_ex_2b,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.48,0.16,0.62 (0.3)
3,-1.00,r_ex_2b_k20_l1_h3_n20,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.39,0.34,0.61 (0.38)
4,-1.00,r_ex_2b_k20_l1_h3_n20,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.31,0.27,0.48 (0.33)
5,-1.00,r_ex_2b_k20_l1_h3_n20,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.48,0.15,0.62 (0.3)
6,0.05,r_ex_2b,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.57,0.29,0.59 (0.37)
7,0.05,r_ex_2b,20,50,37.1,1.05,50,2811,2494.75,50,...,50,123.68,87.29,50,30.1,1.05,50,0.43,0.24,0.46 (0.32)
8,0.05,r_ex_2b,50,50,97.7,1.11,50,2811,2494.75,50,...,50,117.42,82.71,50,78.7,1.11,50,0.53,0.12,0.6 (0.28)
9,0.05,r_ex_2b_k20_l1_h3_n20,10,50,17.5,0.93,50,2811,2494.75,50,...,50,131.30,93.07,50,14.5,0.93,50,0.46,0.31,0.61 (0.38)
